In [9]:
import torch
import numpy as np
import os
from torch import nn
from dataset import CIFAR10_custom
from transforms import CompressedToTensor, ZigZagOrder, ChooseAC, FlattenZigZag
from torchvision.transforms import Compose

torch.set_printoptions(threshold=10_000)
# np.set_printoptions(threshold=np.inf)

In [10]:
download_path = os.path.join('..', 'data', 'cifar10')

if not os.path.exists(download_path):
    os.makedirs(download_path)

In [11]:
transform = Compose([CompressedToTensor(),
                     ZigZagOrder(),
                     ChooseAC(5),
                     FlattenZigZag()])

cifar = CIFAR10_custom(download_path, transform = transform,  download = True)

Files already downloaded and verified


In [12]:
cifar.format

'rgb'

In [13]:
cifar.to_ycbcr(in_place=True)
cifar.compress(in_place=True)

In [14]:
cifar[0][0].shape

torch.Size([3, 96])

In [15]:
cifar[0][0]

tensor([[ 37., -13.,  -7.,   3.,  -7.,  -4.,  42.,   4.,   7.,   0.,  -5.,   1.,
          44.,  -2.,   8.,   0.,  -2.,   4.,  45.,   8.,   9.,   6.,  -5.,  -2.,
          50.,  12.,   3.,  -2.,  -5.,  -1.,  55., -11., -23.,  -2.,  16.,   5.,
          72.,   8., -36.,   3.,   2.,  -1.,  51.,   3.,  -5.,   2.,  -7.,   2.,
          54.,  -6.,  -3.,  -5.,   5.,   8.,  80.,   7.,  11.,   1.,  -4.,   2.,
          73.,  -6.,  23.,   3.,  -2.,   7.,  58.,   5.,   2.,   4.,  12.,  13.,
          65.,   4., -12.,   2.,   1.,   5.,  59.,   1.,   6.,  10.,  -3.,  -1.,
          51.,   4.,  -2.,  -1.,  -1.,  -2.,  53.,  -3.,   8.,   1.,  -3.,  -5.],
        [  2.,  -1.,   0.,   0.,   0.,   0.,   2.,   0.,   0.,   0.,   0.,   0.,
           3.,   0.,   0.,   0.,   0.,   0.,   3.,   0.,   0.,   0.,   0.,   0.,
           3.,   1.,   0.,   0.,   0.,   0.,   3.,  -1.,  -1.,   0.,   1.,   0.,
           4.,   0.,  -2.,   0.,   0.,   0.,   3.,   0.,   0.,   0.,   0.,   0.,
           3.,   0.,   0., 